# Personalities


## Setup


In [32]:
import pandas as pd
from io import StringIO
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import (
    CommaSeparatedListOutputParser,
)

In [33]:
openai_client = OpenAI()
openai_chat = ChatOpenAI(model="gpt-4-turbo", temperature=0)
csv_parser = CommaSeparatedListOutputParser()
csv_format_instructions = csv_parser.get_format_instructions()

## Datasets


In [34]:
personalities_df = pd.read_csv(
    "./datasets/personalities.csv",
    delimiter="|",
    skipinitialspace=True,
)

In [35]:
personality_data = personalities_df[["TestName", "PersonalityType"]].to_csv(sep=",")
personality_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are an expert on personality types. I will send messages about the following personality information. You will provide helpful responses.\n\npersonalities:\n```\n{personality_data}\n```",
        ),
        ("user", "{input}\n\nformat_instructions:{format_instructions}"),
    ]
)
personality_csv_chain = personality_prompt | openai_chat | csv_parser

In [36]:
try:
    leadership_df = pd.read_csv("./datasets/leadership_styles.csv", sep=",")
except FileNotFoundError:
    leadership_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with the leadership style that each PersonalityType prefers to use. In addition to the leadership style, give short and concise 2 sentence descriptions.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, LeadershipStyle, LeadershipDescription",
        }
    )
    leadership_df = pd.read_csv(StringIO("".join(leadership_result)), delimiter=",")
    leadership_df.to_csv("./datasets/leadership_styles.csv", sep=",", index=False)

In [37]:
try:
    communication_df = pd.read_csv("./datasets/communication_styles.csv", sep=",")
except FileNotFoundError:
    communication_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with the communication style that each PersonalityType prefers to use. In addition to the communication style, give short and concise 2 sentence descriptions.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, CommunicationStyle, CommunicationDescription",
        }
    )
    communication_df = pd.read_csv(
        StringIO("".join(communication_result)), delimiter=","
    )
    communication_df.to_csv("./datasets/communication_styles.csv", sep=",", index=False)

In [38]:
try:
    motivators_df = pd.read_csv("./datasets/motivators.csv", sep=",")
except FileNotFoundError:
    motivators_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with the motivators for each PersonalityType. Separate each motivator with a '|' character.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, Motivators",
        }
    )
    motivators_df = pd.read_csv(StringIO("".join(motivators_result)), delimiter=",")
    motivators_df.to_csv("./datasets/motivators.csv", sep=",", index=False)

## Main


In [39]:
personalities_df = personalities_df.merge(leadership_df, on="PersonalityType")
personalities_df = personalities_df.merge(communication_df, on="PersonalityType")
personalities_df = personalities_df.merge(motivators_df, on="PersonalityType")

personalities_df

,TestName,PersonalityType,Archetype,Description,LeadershipStyle,LeadershipDescription,CommunicationStyle,CommunicationDescription,Motivators
0,Myers-Briggs,ISTJ,Inspector,"Responsible organizers, driven to create order...",Directive,ISTJs prefer a structured and orderly approach...,Direct and Detailed,ISTJs prefer clearstraightforward communicatio...,Responsibility|Tradition|Stability|Order
1,Myers-Briggs,ISFJ,Protector,"Dedicated caretakers, eager to help and provid...",Supportive,ISFJs lead with a focus on harmony and coopera...,Warm and Considerate,ISFJs communicate in a warmfriendly mannerofte...,Harmony|Safety|Stability|Service to others
2,Myers-Briggs,INFJ,Counselor,"Insightful visionaries, seeking meaning in rel...",Transformative,INFJs are visionary leaders who focus on the f...,Insightful and Inspiring,INFJs use a thoughtful and often metaphorical ...,Meaningful work|Personal growth|Integrity|Help...
3,Myers-Briggs,INTJ,Mastermind,"Strategic thinkers, determined to improve syst...",Strategic,INTJs prefer to lead by developing long-term s...,Strategic and Logical,INTJs prefer communication that is logical and...,Competence|Independence|Innovation|Achievement
4,Myers-Briggs,ISTP,Craftsman,"Explorers of the physical world, adept at unde...",Adaptive,ISTPs are flexible leaders who excel in crisis...,Analytical and Direct,ISTPs communicate in a straightforward and fac...,Freedom|Variety|Ability to act on logic|Hands-...
5,Myers-Briggs,ISFP,Composer,"Gentle caretakers, living in the moment and en...",Authentic,ISFPs lead by creating an inclusive environmen...,Empathetic and Gentle,ISFPs have a gentle communication styleoften e...,Aesthetic expression|Harmony|Flexibility|Gentl...
6,Myers-Briggs,INFP,Healer,"Idealistic dreamers, guided by core values and...",Inspirational,INFPs lead with empathy and idealismoften advo...,Idealistic and Expressive,INFPs communicate in an idealisticexpressive m...,Idealism|Personal values|Creative expression|A...
7,Myers-Briggs,INTP,Architect,"Inventive thinkers, curious about the world. S...",Analytical,INTPs prefer a leadership style that is logica...,Abstract and Theoretical,INTPs enjoy discussing theories and abstract c...,Intellectual challenge|Competence|Independence...
8,Myers-Briggs,ESTP,Dynamo,"Energetic thrill-seekers, thrive on action. Pr...",Dynamic,ESTPs are action-oriented leaders who thrive o...,Dynamic and Practical,ESTPs use a direct and action-oriented communi...,Action|Impact|Variety|Problem solving
9,Myers-Briggs,ESFP,Performer,"Spontaneous entertainers, enjoying the spotlig...",Expressive,ESFPs lead with enthusiasm and energyoften cre...,Enthusiastic and Spontaneous,ESFPs communicate in a livelyspontaneous manne...,Positive interactions|Fun|Aesthetic experience...
